In [ ]:
import requests
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk import sent_tokenize, word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import re

In [ ]:
Df_pd = pd.read_csv("C:\Users\Shyamal\YT Comments\comments.csv")
Df_pd

In [ ]:
Rev_list = Df_pd["Comment"]

In [ ]:
def preprocess(text):
    clean_data = []
    for x in (text[:]): 
        new_text = re.sub('<.*?>', '', x)   # remove HTML tags
        new_text = re.sub(r'[^\w\s]', '', new_text) # remove punc.
        new_text = re.sub(r'\d+','',new_text)# remove numbers
        new_text = new_text.lower() # lower case, .upper() for upper          
        if new_text != '':
            clean_data.append(new_text)
    return clean_data

In [ ]:
def tokenization_w(words):
    w_new = []
    for w in (words[:]):
        w_token = word_tokenize(w)
        if w_token != '':
            w_new.append(w_token)
    return w_new

In [ ]:
lemmatizer = WordNetLemmatizer()
def lemmatization(words):
    new = []
    for i in range(len(Rev_list)):
        lem_words = [lemmatizer.lemmatize(x) for x in (words[:][i])]
        new.append(lem_words)
    return new

In [ ]:
clean_list = preprocess(Rev_list) #removes punctuation, see above
clean_words = tokenization_w(clean_list) # word tokenization
lem = lemmatization(clean_words)
lem

In [ ]:
new_comment = []
for i in lem:
    new_comment.append(" ".join(i))
new_comment

In [ ]:
data = pd.DataFrame({'comment':new_comment})
data['review']

In [ ]:
data['word_count'] = data['comment'].apply(lambda x: len(str(x).split(" ")))
data[['comment','word_count']].head()

In [ ]:
data['char_count'] = data['review'].str.len() ## this also includes spaces
data[['comment','char_count']].head()

In [ ]:
#Average Word Length
#Number of characters(without space count)/Total number of words
def avg_word(sentence):
  words = sentence.split()
  print(words)
  print(len(words))
  print(sum(len(word) for word in words))
  if words == []:
    return 0
  return (sum(len(word) for word in words)/len(words))

data['avg_word'] = data['comment'].apply(lambda x: avg_word(x))
data[['comment','avg_word']].head()

In [ ]:
#Number of stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

data['stopwords'] = data['comment'].apply(lambda x: len([x for x in x.split() if x in stop]))
data[['comment','stopwords']].head()

In [ ]:
#Number of special characters
data['hastags'] = data['comment'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))
data[['comment','hastags']].head()

In [ ]:
#Number of numerics
data['numerics'] = data['comment'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
data[['comment','numerics']].head()

In [ ]:
#Number of Uppercase words
data['upper'] = data['comment'].apply(lambda x: len([x for x in x.split() if x.isupper()]))
data[['comment','upper']].head()

In [ ]:
pos_family = {
    'noun' : ['NN','NNS','NNP','NNPS'],
    'pron' : ['PRP','PRP$','WP','WP$'],
    'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
    'adj' :  ['JJ','JJR','JJS'],
    'adv' : ['RB','RBR','RBS','WRB']
}

# function to check and get the part of speech tag count of a words in a given sentence
from textblob import TextBlob, Word, Blobber
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
def check_pos_tag(x, flag):
    cnt = 0
    try:
        wiki = TextBlob(x)
        for tup in wiki.tags:
            ppo = list(tup)[1]
            if ppo in pos_family[flag]:
                cnt += 1
                print(ppo, tup)
    except:
        pass
    return cnt

data['noun_count'] = data['comment'].apply(lambda x: check_pos_tag(x, 'noun'))
data['verb_count'] = data['comment'].apply(lambda x: check_pos_tag(x, 'verb'))
data['adj_count'] = data['comment'].apply(lambda x: check_pos_tag(x, 'adj'))
data['adv_count'] = data['comment'].apply(lambda x: check_pos_tag(x, 'adv'))
data['pron_count'] = data['comment'].apply(lambda x: check_pos_tag(x, 'pron'))
data[['comment','noun_count','verb_count','adj_count', 'adv_count', 'pron_count' ]].head()


In [ ]:
data.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
cv=CountVectorizer()
A_vec = cv.fit_transform(new_review)
print(A_vec.toarray())

In [ ]:
tv=TfidfVectorizer()
t_vec = tv.fit_transform(new_review)
print(t_vec.toarray())

In [ ]:
feature_names = tv.get_feature_names()

dense = t_vec.todense()
denselist = dense.tolist()
daf = pd.DataFrame(denselist, columns=feature_names)

In [ ]:
feature_names

In [ ]:
df_c =pd.concat([daf,data], axis=1)
df_c.head()